# Step 1: Load the data


In [1]:
!wget https://files.webis.de/corpora/corpora-webis/known-item-question-performance-prediction/with-term-recall-in-progress/training-data-main-06-07-2023.jsonl.gz

--2023-06-15 07:09:13--  https://files.webis.de/corpora/corpora-webis/known-item-question-performance-prediction/with-term-recall-in-progress/training-data-main-06-07-2023.jsonl.gz
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56442287 (54M) [application/octet-stream]
Saving to: ‘training-data-main-06-07-2023.jsonl.gz’

training-data-main- 100%[===================>]  53.83M  12.4MB/s    in 12s     

2023-06-15 07:09:26 (4.47 MB/s) - ‘training-data-main-06-07-2023.jsonl.gz’ saved [56442287/56442287]



In [2]:
!cat training-data-main-06-07-2023.jsonl.gz |gunzip > training-data-main-06-07-2023.jsonl

In [3]:
!head -3 training-data-main-06-07-2023.jsonl

{"hide_score": false, "domain": "self.tipofmytongue", "author_flair_text": null, "created_utc": "1409621706", "solved_utc": "1409621904", "subreddit": "tipofmytongue", "media_embed": {}, "secure_media_embed": {}, "chosen_answer": "[Wayside School (book series)](http://www.louissachar.com/Wayside.htm)", "links_on_answer_path": ["http://www.louissachar.com/Wayside.htm)"], "title": "[TOMT][BOOK] Book I read in 3rd grade", "thumbnail": "self", "downs": 0.0, "distinguished": null, "link_flair_text": "Solved", "permalink": "/r/tipofmytongue/comments/2f7wg1/tomtbook_book_i_read_in_3rd_grade/", "ups": 0.0, "retrieved_on": 1441191246.0, "content": "There was a book I read in third grade... here I what I remember:\n\nThe book was about a class of students who went to a school with like thirty floors... Except one floor was missing. That is basically all I remember. Except, each chapter focused around a different student/teacher.", "media": null, "link_flair_css_class": "solvedcase", "name": "t3_

In [9]:
import json
import pandas as pd

def to_simpletransformer_seq_2_seq_format(entries):
    ret = []
    for entry in entries:
        if 'training_samples' in entry and 'instances' in entry['training_samples'] and entry['training_samples']['instances']:
            for instance in entry['training_samples']['instances']:
              ret += [{'input_text': instance['doc']['title'], 'target_text': ' '.join(instance['term_recall'].keys())}]
    return ret

df_train = [json.loads(i) for i in open('training-data-main-06-07-2023.jsonl')]
df_train = to_simpletransformer_seq_2_seq_format(df_train)
df_train = pd.DataFrame(df_train)


df_train = df_train[:1000]

df_eval = df_train[800:]
df_train = df_train[:800]
df_train

,input_text,target_text
0,Book I read in 3rd grade. There was a book I ...,school floors read book floor teacher one
1,where one of the strips starts as a casual lo...,art first starts looking ago well year one
2,Name of song in this video. https://www.youtu...,watch www song https video v youtube com
3,1990's Educational Cartoon for kids to learn ...,count featured 3 time watching
4,A woman birthing monsters/aliens. I was re-wa...,like birth aliens time giving scene watching b...
...,...,...
795,"On SNL, there’s an office party sketch where ...",one
796,Singer who just hits it big and then loses hi...,becomes sister recent movies girl died hits one
797,"A movie about a guy's life, his school, marri...",gets leave narrating university 2 watching fri...
798,A guy showing his girlfriend/wife what it’s l...,showing key wife tried bucket hose aiming


In [24]:
df_train.iloc[0].to_dict()

{'input_text': ' Book I read in 3rd grade. There was a book I read in third grade... here I what I remember:\n\nThe book was about a class of students who went to a school with like thirty floors... Except one floor was missing. That is basically all I remember. Except, each chapter focused around a different student/teacher.',
 'target_text': 'school floors read book floor teacher one'}

In [10]:
df_eval

,input_text,target_text
800,". Hello, hopefully someone will be able to hel...",c
801,Looking for an educational videogame for kids...,clouds solve games game math
802,"educational game, with some robot?. I remembe...",main like enter robot building played colors e...
803,An article I saw on r/politics about the CEO?...,trying saying article news politics president ...
804,Striking blonde hosts annual? party and innoc...,like mansion years young r man end www source ...
...,...,...
995,In a Dream Called Love. I was in a hooka loun...,remember song dream came please
996,"Graph of Eminem's ""Lose Yourself""?. Hi guys. ...",funny like lose hope line song guys known help...
997,"10,000 hours, or how many chances you have to...",showed years malcolm 10 average hours takes he...
998,"Social Psychology term for ""I know that you k...",new video 3 v http watching com animated watch...


In [22]:
df_eval.iloc[10:15].to_dict()

{'input_text': {810: 'NEED HELP TO FIND A CHILDHOOD CARTOONS! I FORGOT THE NAME 3. about water ice and gas(shaped as cloud ) they hang with each other',
  811: ' Iron Maiden riff that reminds me of some NU metal song. [The riff at 1:46 in Genghis Khan](http://youtu.be/6PHAwWUx21U?t=01m46s) reminds me of another song, possibly Nu metal, from the early 2000s. Thoughts?\n\nEdit: I’m stupid, there are dozens of people talking about it in the comments, I didn’t even think to check as I assumed it was an obscure observation',
  812: ' Hand-held "Cars" game console. I vaguely remember playing with a relatively large hand-held Pixar\'s "Cars" themed game console in my room as a child. It was designed to look like a radio and it had black and green (or white) Game Boy-esque 8-bit graphics, and you could choose from a variety of basic "Cars" themed minigames. Did anyone else have one of these? I also remember listening to a "Cars" song about highways or something that mentioned some of the Unite

# Step 2: Filter the data or create variants

E.g., try to find and remove cases where the training data looks odd:

```
{"doc": {"title": " Name of song in this video. https://www.youtube.com/watch?v=HpSs2Zt5Fz4", "position": 1, "id": 1}, "term_recall": {"watch": 1.0, "www": 1.0, "song": 1.0, "https": 1.0, "video": 1.0, "v": 1.0, "youtube": 1.0, "com": 1.0}}
```


In [ ]:
#tbd

# Step 3: Train Model(s)

In [6]:
!pip3 install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.

In [7]:
import logging

import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [11]:
# Configure the model
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 2
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

# Weitere Modelle die wir probieren sollten: T5, BART, ?
#model = Seq2SeqModel(
#    encoder_decoder_type="bart",
#    encoder_decoder_name="facebook/bart-large",
#    args=model_args,
#)


model = Seq2SeqModel(
    "roberta",
    "roberta-base",
    "bert-base-cased",
    args=model_args,
)

# Train the model
!rm -Rf outputs/
model.train_model(df_train, eval_data=df_eval)

# Evaluate the model
result = model.eval_model(df_eval)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkp

  0%|          | 0/800 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learnin

...

Generating outputs:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Generating outputs:   0%|          | 0/25 [00:00<?, ?it/s]

In [15]:
model = Seq2SeqModel(
    "roberta",
    "roberta-base",
    "bert-base-cased",
    encoder_decoder_name="outputs/checkpoint-200-epoch-2",
    args=model_args,
)

model.predict([
    'Book I read in 3rd grade. There was a book I read in third grade... here I what I remember:\n\nThe book was about a class of students who went to a school with like thirty floors... Except one floor was missing. That is basically all I remember. Except, each chapter focused around a different student/teacher.',
    """1990's Educational Cartoon for kids to learn French. Hi all,\n\nWhen I was really young, 3-5, I remember watching a cartoon that I think was supposed to teach kids french. I would guess it was made from 1990-1995, but possibly earlier.\n\nIt was in color and the episodes I remember featured a guy with with a long, narrow, and crooked nose and greenish skin teaching kids how to count? There was also a scene that had some character running up a clock tower to change the time.\n\nOverall it was a pretty gloomy feel, iirc, and I'd love to see it again if possible""",
])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['', '']

In [16]:
# Configure the model
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 2
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.output_dir = 'model-bart/'

# Weitere Modelle die wir probieren sollten: T5, BART, ?
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)


# Train the model
!rm -Rf model-bart/
model.train_model(df_train, eval_data=df_eval)

# Evaluate the model
result = model.eval_model(df_eval)



  0%|          | 0/800 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/25 [00:00<?, ?it/s]

In [23]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="model-bart/checkpoint-200-epoch-2",
    args=model_args,
)

model.predict([
    'Book I read in 3rd grade. There was a book I read in third grade... here I what I remember:\n\nThe book was about a class of students who went to a school with like thirty floors... Except one floor was missing. That is basically all I remember. Except, each chapter focused around a different student/teacher.',
    """1990's Educational Cartoon for kids to learn French. Hi all,\n\nWhen I was really young, 3-5, I remember watching a cartoon that I think was supposed to teach kids french. I would guess it was made from 1990-1995, but possibly earlier.\n\nIt was in color and the episodes I remember featured a guy with with a long, narrow, and crooked nose and greenish skin teaching kids how to count? There was also a scene that had some character running up a clock tower to change the time.\n\nOverall it was a pretty gloomy feel, iirc, and I'd love to see it again if possible""",


    "NEED HELP TO FIND A CHILDHOOD CARTOONS! I FORGOT THE NAME 3. about water ice and gas(shaped as cloud ) they hang with each other",
    "Iron Maiden riff that reminds me of some NU metal song. [The riff at 1:46 in Genghis Khan](http://youtu.be/6PHAwWUx21U?t=01m46s) reminds me of another song, possibly Nu metal, from the early 2000s. Thoughts?\n\nEdit: I’m stupid, there are dozens of people talking about it in the comments, I didn’t even think to check as I assumed it was an obscure observation",
    """ Hand-held "Cars" game console. I vaguely remember playing with a relatively large hand-held Pixar\'s "Cars" themed game console in my room as a child. It was designed to look like a radio and it had black and green (or white) Game Boy-esque 8-bit graphics, and you could choose from a variety of basic "Cars" themed minigames. Did anyone else have one of these? I also remember listening to a "Cars" song about highways or something that mentioned some of the United States (including Tennessee), but I don\'t remember if that was a separate thing or not.""",
   """Chinese Documentary about a Brat Camp. I once watched a Chinese documentary about a Brat Camp.  According to the documentary the 1 child policy created a whole generation of spoiled kids because of the fact that they were the only child.  Unless I misunderstood the camp consisted of forced hikes for the kids which sounds easy on paper but eventually it got really tiring for them.""",
])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['trying one floor',
 'trying time animated animated animated 3-5 animated time',
 'like',
 'song 1',
 'like game car game',
 'movie 1 child kids 1 child child']

In [13]:
!ls outputs/

best_model		decoder		  model_args.json
checkpoint-100-epoch-1	encoder		  training_args.bin
checkpoint-200-epoch-2	eval_results.txt  training_progress_scores.csv


In [ ]:
model.eval_model(df_eval)

  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Generating outputs:   0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 8.584575542045059e-05}

In [ ]:
model.predict([
    'Book I read in 3rd grade. There was a book I read in third grade... here I what I remember:\n\nThe book was about a class of students who went to a school with like thirty floors... Except one floor was missing. That is basically all I remember. Except, each chapter focused around a different student/teacher.',
    """1990's Educational Cartoon for kids to learn French. Hi all,\n\nWhen I was really young, 3-5, I remember watching a cartoon that I think was supposed to teach kids french. I would guess it was made from 1990-1995, but possibly earlier.\n\nIt was in color and the episodes I remember featured a guy with with a long, narrow, and crooked nose and greenish skin teaching kids how to count? There was also a scene that had some character running up a clock tower to change the time.\n\nOverall it was a pretty gloomy feel, iirc, and I'd love to see it again if possible""",
])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['', '']

In [ ]:
model_untrained = Seq2SeqModel(
    "roberta",
    "roberta-base",
    "bert-base-cased",
    args=model_args,
)

model_untrained.eval_model(df_eval)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another a

  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Generating outputs:   0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 8.733285387039185}

In [ ]:
model_untrained.predict([
    'Book I read in 3rd grade. There was a book I read in third grade... here I what I remember:\n\nThe book was about a class of students who went to a school with like thirty floors... Except one floor was missing. That is basically all I remember. Except, each chapter focused around a different student/teacher.',
    """1990's Educational Cartoon for kids to learn French. Hi all,\n\nWhen I was really young, 3-5, I remember watching a cartoon that I think was supposed to teach kids french. I would guess it was made from 1990-1995, but possibly earlier.\n\nIt was in color and the episodes I remember featured a guy with with a long, narrow, and crooked nose and greenish skin teaching kids how to count? There was also a scene that had some character running up a clock tower to change the time.\n\nOverall it was a pretty gloomy feel, iirc, and I'd love to see it again if possible""",
])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['he he he he he he he he he he he he he he he he he he he',
 'he he he he he he he he he he he he he he he he he he he']

In [ ]:
model.predict(
        [
            "Perseus “Percy” Jackson is the main protagonist and the narrator of the Percy Jackson and the Olympians series.",
        ])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['- - - - - - - - - - - - - - - - - - -']